![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [7]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
import time

filename_features = "/content/features.txt"
filename_labels = "/content/activity_labels.txt"

filename_subtrain = "/content/subject_train.txt"
filename_xtrain = "/content/X_train.txt"
filename_ytrain = "/content/y_train.txt"

filename_subtest = "/content/subject_test.txt"
filename_xtest = "/content/X_test.txt"
filename_ytest = "/content/y_test.txt"

# Carregar features e labels
features = pd.read_csv(filename_features, sep='\s+', header=None, names=['idx', 'nome_var'], encoding="utf-8")
features = features['nome_var'].tolist()

# Remover duplicatas dos nomes das variáveis
features = list(dict.fromkeys(features))  # Remove duplicatas mantendo a ordem

labels = pd.read_csv(filename_labels, sep='\s+', header=None, names=['cod_label', 'label'], encoding="utf-8")

# Carregar dados de treino
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], encoding="utf-8")['subject_id']
X_train = pd.read_csv(filename_xtrain, sep='\s+', header=None, names=features, encoding="utf-8")
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'], encoding="utf-8")

# Carregar dados de teste
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], encoding="utf-8")['subject_id']
X_test = pd.read_csv(filename_xtest, sep='\s+', header=None, names=features, encoding="utf-8")
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'], encoding="utf-8")

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [6]:
%%time
# Inicializar o classificador com o parâmetro ccp_alpha
clf = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001)

# Medir o tempo de treinamento
start_train = time.time()
clf.fit(X_train, y_train)
end_train = time.time()

# Fazer previsões no conjunto de treino e teste
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Medir o tempo de predição no conjunto de teste
start_test = time.time()
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
end_test = time.time()

# Exibir os resultados
print("Tempo de treinamento: {:.4f} segundos".format(end_train - start_train))
print("Tempo de teste: {:.4f} segundos".format(end_test - start_test))
print("Acurácia no conjunto de treinamento: {:.4f}".format(accuracy_train))
print("Acurácia no conjunto de teste: {:.4f}".format(accuracy_test))

Tempo de treinamento: 6.9027 segundos
Tempo de teste: 0.0052 segundos
Acurácia no conjunto de treinamento: 0.9763
Acurácia no conjunto de teste: 0.8789
CPU times: user 6.76 s, sys: 6.31 ms, total: 6.76 s
Wall time: 6.92 s


### Avaliação de Desempenho do Modelo

#### Resultados:
- **Tempo de Processamento:**
  - **Tempo de Treinamento:** 6.9027 segundos
  - **Tempo de Teste:** 0.0052 segundos

- **Acurácia:**
  - **Conjunto de Treinamento:** 0.9763 (bom ajuste, sem sinais evidentes de overfitting extremo)
  - **Conjunto de Teste:** 0.8789 (boa capacidade de generalização)

- **CPU Times:**
  - **User Time:** 6.76 s
  - **System Time:** 6.31 ms
  - **Wall Time:** 6.92 s

---

### Observações:
1. **Eficiência Computacional:**  
   O treinamento consumiu um tempo razoável (6.9027 segundos), considerando o modelo e o tamanho dos dados. O tempo de teste foi extremamente rápido (0.0052 segundos), o que demonstra eficiência nas previsões.

2. **Desempenho Geral:**  
   A acurácia no conjunto de treinamento (97.63%) mostra que o modelo ajustou-se bem aos padrões dos dados, enquanto a acurácia no conjunto de teste (87.89%) sugere uma boa capacidade de generalização. A diferença entre as duas acurácias não é grande, o que indica que o modelo está balanceado.

3. **Interpretação Geral:**  
   Uma acurácia de 87.89% no conjunto de teste é sólida e mostra que o modelo consegue capturar padrões relevantes, embora ainda haja espaço para pequenas melhorias.

---


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [8]:
%%time
# 1. Aplicar PCA nas variáveis originais
pca = PCA(n_components=1)  # Reduzir para apenas 1 componente principal
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# 2. Treinar a árvore de decisão com a componente principal como variável explicativa
clf = DecisionTreeClassifier(random_state=42)
start_train = time.time()
clf.fit(X_train_pca, y_train)
end_train = time.time()

# 3. Fazer previsões nos conjuntos de treinamento e teste
y_train_pred = clf.predict(X_train_pca)
y_test_pred = clf.predict(X_test_pca)

# 4. Avaliar a acurácia
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

# 5. Medir o tempo de processamento
print("Tempo de treinamento: {:.4f} segundos".format(end_train - start_train))
print("Acurácia no conjunto de treinamento: {:.4f}".format(accuracy_train))
print("Acurácia no conjunto de teste: {:.4f}".format(accuracy_test))

Tempo de treinamento: 0.0763 segundos
Acurácia no conjunto de treinamento: 0.9999
Acurácia no conjunto de teste: 0.3777
CPU times: user 585 ms, sys: 2.52 ms, total: 588 ms
Wall time: 688 ms


### Avaliação de Desempenho do Modelo

#### Resultados:
- **Tempo de Treinamento:**  
  - **0.0763 segundos**  
  O treinamento foi extremamente rápido, como esperado para uma árvore de decisão com uma única variável explicativa.

- **Acurácia:**
  - **Conjunto de Treinamento:**  
    - **0.9999**  
    A acurácia foi quase perfeita no conjunto de treinamento, indicando ajuste completo aos dados de treinamento (**overfitting** pode ser uma preocupação).

  - **Conjunto de Teste:**  
    - **0.3777**  
    A acurácia no conjunto de teste foi significativamente menor, sugerindo que o modelo não conseguiu generalizar bem para novos dados.

- **Tempo Total de Processamento:**  
  - **CPU Times:** 585 ms  
  - **Wall Time:** 688 ms  
  O tempo total de processamento foi eficiente, considerando o treinamento, a aplicação do PCA e as predições.

---

### Observações:
1. **Tempo de Treinamento:**  
   O modelo foi treinado rapidamente, um indicador de boa eficiência computacional.

2. **Overfitting:**  
   O alto desempenho no treinamento e baixo desempenho no teste indicam que o modelo pode ter ajustado excessivamente aos dados de treinamento.

3. **Informações Limitadas:**  
   Usar apenas uma componente principal provavelmente não foi suficiente para capturar a variância total dos dados, o que impactou negativamente o desempenho no conjunto de teste.

---


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [10]:
%%time
# Lista de números de componentes para teste
n_components_list = [1, 2, 5, 10, 50]

# Lista para armazenar os resultados
results = []

for n in n_components_list:
    # Aplicar PCA com n componentes
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Treinar a árvore de decisão
    clf = DecisionTreeClassifier(random_state=42)
    start_train = time.time()
    clf.fit(X_train_pca, y_train)
    end_train = time.time()

    # Fazer previsões e medir acurácia
    start_test = time.time()
    y_train_pred = clf.predict(X_train_pca)
    y_test_pred = clf.predict(X_test_pca)
    end_test = time.time()

    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)

    # Armazenar resultados em um dicionário
    results.append({
        'n_components': n,
        'tempo_treinamento': end_train - start_train,
        'tempo_teste': end_test - start_test,
        'acuracia_treinamento': accuracy_train,
        'acuracia_teste': accuracy_test
    })

# Converter a lista de resultados em um DataFrame
results_df = pd.DataFrame(results)

# Exibir os resultados
print(results_df)

   n_components  tempo_treinamento  tempo_teste  acuracia_treinamento  \
0             1           0.141987     0.011818              0.999864   
1             2           0.074515     0.003815              1.000000   
2             5           0.127085     0.003152              1.000000   
3            10           0.241316     0.003009              1.000000   
4            50           0.802395     0.003564              1.000000   

   acuracia_teste  
0        0.377672  
1        0.451985  
2        0.617238  
3        0.763488  
4        0.762470  
CPU times: user 4.26 s, sys: 28.2 ms, total: 4.29 s
Wall time: 3.59 s


### Avaliação de Desempenho do Modelo com Diferentes Quantidades de Componentes

#### Resultados:
| **n_components** | **Tempo de Treinamento (s)** | **Tempo de Teste (s)** | **Acurácia Treinamento** | **Acurácia Teste** |
|-------------------|-----------------------------|-------------------------|---------------------------|--------------------|
| 1                 | 0.141987                   | 0.011818               | 0.999864                 | 0.377672           |
| 2                 | 0.074515                   | 0.003815               | 1.000000                 | 0.451985           |
| 5                 | 0.127085                   | 0.003152               | 1.000000                 | 0.617238           |
| 10                | 0.241316                   | 0.003009               | 1.000000                 | 0.763488           |
| 50                | 0.802395                   | 0.003564               | 1.000000                 | 0.762470           |

#### Processamento Total:
- **CPU Times:** 4.26 s (user), 28.2 ms (sys), 4.29 s (total)
- **Wall Time:** 3.59 s

---

### Observações:
1. **Tempo de Treinamento:**
   - O tempo de treinamento aumenta com o número de componentes principais, o que é esperado, dado que a quantidade de dados a ser processada cresce.

2. **Tempo de Teste:**
   - O tempo de teste permanece baixo e estável independentemente do número de componentes, o que demonstra eficiência para predições.

3. **Acurácia no Treinamento:**
   - A acurácia no treinamento é consistentemente alta (praticamente perfeita), indicando que o modelo ajusta bem aos dados de treinamento.

4. **Acurácia no Teste:**
   - A acurácia no teste melhora com o aumento no número de componentes até cerca de 10, com uma leve redução ao usar 50 componentes, sugerindo uma possível saturação ou overfitting.

---

### Interpretação:
- **Componentes Otimizados:** Usar 10 componentes parece oferecer o melhor equilíbrio entre acurácia no conjunto de teste (76.35%) e tempo de treinamento.
- **Limitação com Poucas Componentes:** Com apenas 1 ou 2 componentes, a acurácia no conjunto de teste é baixa, indicando que essas dimensões não capturam variância suficiente para discriminar os dados.

---


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

### Conclusão

#### O que aconteceu com a acurácia?
- À medida que o número de componentes principais aumentou, a acurácia no conjunto de teste **melhorou significativamente até 10 componentes**, atingindo aproximadamente **76.35%**.
- Contudo, ao usar 50 componentes, a acurácia no teste apresentou **uma leve redução para 76.25%**, possivelmente devido a **saturação dos dados ou overfitting**, pois o modelo começa a considerar informações menos relevantes.

#### O que aconteceu com o tempo de processamento?
- **Tempo de Treinamento:** O tempo de treinamento **aumentou com o número de componentes principais**, de 0.14 segundos para 0.80 segundos. Esse aumento reflete o processamento adicional necessário para trabalhar com mais dimensões nos dados.
- **Tempo de Teste:** O tempo de teste permaneceu **estável e muito baixo** (~0.003 segundos em média), mostrando que o modelo é eficiente para predições, independentemente do número de componentes.

---

### Insights:
- **Melhor Número de Componentes:** 10 componentes oferecem o melhor equilíbrio entre **acurácia** (76.35%) e **tempo de processamento**.
- **Poucos Componentes (1 ou 2):** Produzem baixa acurácia no teste (< 45%), indicando que a variância capturada não é suficiente para representar os dados.
- **Componentes Demais (50):** Aumento no tempo de treinamento sem ganhos significativos na acurácia, sugerindo que a partir de certo ponto, componentes adicionais trazem pouca ou nenhuma vantagem.

